## Extending plots with widgets

This activity will combine most of what you have already learned about Bokeh. You will also need the skills you have acquired while working with Pandas.  

We will create an interactive visualization that displays a candle stick plot, which is often used when handling stock price data.   
We will be able to compare two stocks with each other by selecting them from dropdowns.   
A RangeSlider will allow us to restrict the displayed date range in the requested year 2016.   
Depending on what graph we choose, we will either see the candle stick visualization or a simple line plot displaying the volume of the selected stock.

The dataset of this exercise contains temporal stock price data.   
This means we'll be looking at data over a range of time.   
> The dataset for this exercise is quite big and has to be downloaded and added into the data folder.   
It can be downloaded from here:   
https://www.kaggle.com/dgawlik/nyse#prices.csv

#### Loading our dataset

In [1]:
# importing the necessary dependencies
import numpy as np
import pandas as pd

In [2]:
# make bokeh display figures inside the notebook
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [3]:
# loading the Dataset with geoplotlib
dataset = pd.read_csv('./data/stock_prices.csv')

In [4]:
# looking at the dataset
dataset.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


Just as in the previous exercise, we want to map the date column to another column with the shortened date that only contains the year, month, and day.

In [5]:
# mapping the date of each row to only the year-month-day format
from datetime import datetime

def shorten_time_stamp(timestamp):
    shortened = timestamp[0]
    
    if len(shortened) > 10:
        parsed_date=datetime.strptime(shortened, '%Y-%m-%d %H:%M:%S')
        shortened=datetime.strftime(parsed_date, '%Y-%m-%d')
    
    return shortened

dataset['short_date'] = dataset.apply(lambda x: shorten_time_stamp(x), axis=1)

**Note:**   
The exectuion of the cell will take a moment since it's a fairly large dataset.   
Please be patient.

In [6]:
# looking at the dataset with shortened dat
dataset.head()

,date,symbol,open,close,low,high,volume,short_date
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0,2016-01-05
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0,2016-01-06
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0,2016-01-07
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0,2016-01-08
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0,2016-01-11


**Note:**   
The last, newly added, column now holds the timestamp without the hour, minute, and second information.   

---

#### Building an interactive visualization

... TODO

In [7]:
# importing the necessary dependencies 
from bokeh.plotting import figure, show
from ipywidgets import interact, widgets

...TODO   
https://bokeh.pydata.org/en/latest/docs/gallery/candlestick.html
https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/legends.html

In [192]:
def add_candle_plot(plot, stock_name, stock_range, color):
    inc_1 = stock_range.close > stock_range.open
    dec_1 = stock_range.open > stock_range.close
    w = 0.5

    plot.segment(stock_range['short_date'], stock_range['high'], 
                 stock_range['short_date'], stock_range['low'], 
                 color="grey")

    plot.vbar(stock_range['short_date'][inc_1], w, 
              stock_range['high'][inc_1], stock_range['close'][inc_1], 
              fill_color="green", line_color="black",
              legend=('Mean price of ' + stock_name), muted_alpha=0.2)

    plot.vbar(stock_range['short_date'][dec_1], w, 
              stock_range['high'][dec_1], stock_range['close'][dec_1], 
              fill_color="red", line_color="black",
              legend=('Mean price of ' + stock_name), muted_alpha=0.2)

    stock_mean_val=stock_range[['high', 'low']].mean(axis=1)
    plot.line(stock_range['short_date'], stock_mean_val, 
              legend=('Mean price of ' + stock_name), muted_alpha=0.2,
              line_color=color, alpha=0.5)

    return plot

In [ ]:
...TODO

In [193]:
# method to build the plot
def get_plot(stock_1, stock_2, date, value):    
    stock_1 = dataset[dataset['symbol'] == stock_1]
    stock_2 = dataset[dataset['symbol'] == stock_2]
    
    stock_1_name=stock_1['symbol'].unique()[0]
    stock_1_range=stock_1[(stock_1['short_date'] >= date[0]) & (stock_1['short_date'] <= date[1])]
    stock_2_name=stock_2['symbol'].unique()[0]
    stock_2_range=stock_2[(stock_2['short_date'] >= date[0]) & (stock_2['short_date'] <= date[1])]

    plot=figure(title='Stock prices', 
                     x_axis_label='Date', 
                     x_range=stock_1_range['short_date'], 
                     y_axis_label='Price in $USD',
                     plot_width=800, 
                     plot_height=500)
    
    plot.xaxis.major_label_orientation = 1
    plot.grid.grid_line_alpha=0.3
    
    if value == 'open-close':
        candle_plot_1=add_candle_plot(plot, stock_1_name, stock_1_range, 'blue')
        candle_plot_2=add_candle_plot(plot, stock_2_name, stock_2_range, 'orange')
        
    if value == 'volume':
        plot.line(stock_1_range['short_date'], stock_1_range['volume'], 
                  legend=stock_1_name, muted_alpha=0.2)
        plot.line(stock_2_range['short_date'], stock_2_range['volume'], 
                  legend=stock_2_name, muted_alpha=0.2,
                  line_color='orange')
    
    plot.legend.click_policy="mute"
    
    return plot


...TODO

In [194]:
# extracing all the stock names
stock_names=dataset['symbol'].unique()
dates_2016=dataset[dataset['short_date'] >= '2016-01-01']['short_date']
unique_dates_2016=sorted(dates_2016.unique())
value_options=['open-close', 'volume']

...TODO

**Note:**   
As mentioned in the previous exercise, we can also make use of the widgets described here: https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html

In [195]:
# setting up the interaction elements
drp_1=widgets.Dropdown(options=stock_names,
                       value='AAPL',
                       description='Compare:')

drp_2=widgets.Dropdown(options=stock_names,
                       value='AON',
                       description='to:')

range_slider=widgets.SelectionRangeSlider(options=unique_dates_2016, 
                                          index=(0,25), 
                                          continuous_update=False,
                                          description='From-To',
                                          layout={'width': '500px'})

value_radio=widgets.RadioButtons(options=value_options,
                                 value='open-close',
                                 description='Metric',
                                 orientation='horizontal')

...TODO

In [196]:
# creating the interact method 
@interact(stock_1=drp_1, stock_2=drp_2, date=range_slider, value=value_radio)
def get_stock_for_2016(stock_1, stock_2, date, value):
    show(get_plot(stock_1, stock_2, date, value))

interactive(children=(Dropdown(description='Compare:', index=4, options=('WLTW', 'A', 'AAL', 'AAP', 'AAPL', 'A…

**Note:**   
We can already see that each date is displayed on the x-axis. If we want to display a bigger time range, we have to customize the ticks on our x-axis. This can be done using `Ticker` objects.   